<h1>Naive bayes cassification</h1>
<h3>Here I import the libraries I need</h3>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

<h3>Here I read data and drop the unnessecary column (id)</h3>

In [2]:
data = pd.read_csv('./data.csv')
data = data.drop(columns=['id'])

<h3>Here I split test/train data and calculate population of each class and their probablities in train data</h3>

In [3]:
classM, classB = 0,0
X = data.drop(columns=["diagnosis"])
y = data["diagnosis"]
X.drop(columns=['Unnamed: 32'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
row_count = len(y_train)
for row in y_train:
    if row == "M":
        classM += 1
    else :
        classB += 1
prob_classM = classM / row_count
prob_classB = classB / row_count
print(prob_classM, prob_classB)

0.37142857142857144 0.6285714285714286


<h3>Here I split the numeric ones as just the label is not numeric I won't lose any data. And also I calculate Q1,Q2,Q3 on data so I can use them as boundaries and split the data into 4 parts that I can easily implement Naive Bayes on</h3>

In [4]:
X_train = X_train.fillna(X_train.median())
Q1 = X_train.quantile(0.25)
Q2 = X_train.median()
Q3 = X_train.quantile(0.75)

<h3>This function calculates the probablity of an attribute value on given class based on attribute value of data</h3>

In [5]:
def calc(key, value, class_label):
    count = 0
    for i in range(len(X_train)):
        if y_train.iloc[i] == class_label:
            if value == 1:
                if X_train[key].iloc[i] <= Q1[key]:
                    count += 1
            elif value == 2:
                if Q1[key] < X_train[key].iloc[i] <= Q2[key]:
                    count += 1
            elif value == 3:
                if Q2[key] < X_train[key].iloc[i] <= Q3[key]:
                    count += 1
            else:
                if X_train[key].iloc[i] > Q3[key]:
                    count += 1
    if count == 0:
        if class_label == 'M':
            return 1/classM+4
        return 1/classB+4
    if class_label == 'M':
        return count/classM
    return count/classB

<h3>This function gets all calculation of attributes of given data and calculates the probablity based on Naive Bayes formula for each class</h3>

In [6]:
def predict(data):
    cM = classM
    cB = classB
    for key, value in Q1.items():
        if data[key] > Q3[key]:
            val = 4
        elif Q2[key] < data[key] <= Q3[key]:
            val = 3
        elif Q1[key] < data[key] <= Q2[key]:
            val = 2
        else:
            val = 1
        cB *= calc(key, val, 'B')
        cM *= calc(key, val, 'M')
    print(f'result for  data is {cM} for classM and {cB} for classB')
    if cM < cB:
        return 'B'
    return 'M'

<h3>Here we get actual labels and predicted ones</h3>

In [7]:
predict = np.array([predict(row) for _, row in X_test.iterrows()])

result for  data is 1.5056076061492387e-20 for classM and 5.846995232737851e-16 for classB
result for  data is 8.078053119781554e-13 for classM and 3.5836150916588405e-30 for classB
result for  data is 1.0677764177889662e-12 for classM and 1.5509394801876156e-26 for classB
result for  data is 1.914570561372913e-24 for classM and 6.176798923744045e-16 for classB
result for  data is 5.549028486969036e-26 for classM and 1.0622155573501196e-14 for classB
result for  data is 2.8581981477056395e-08 for classM and 2.841071973440571e-35 for classB
result for  data is 7.684494219425603e-10 for classM and 6.600613341106734e-34 for classB
result for  data is 8.230276880596325e-14 for classM and 1.7096206300920104e-28 for classB
result for  data is 2.4390812209469557e-18 for classM and 1.0255889616910188e-20 for classB
result for  data is 4.915771609246367e-28 for classM and 4.4558758148062803e-14 for classB
result for  data is 1.229524828196868e-31 for classM and 1.631211920988562e-13 for classB


<h3>Here we calculate confusion matrix. TP is for class 'M' and TN is for class 'B'</h3>

In [8]:
TP, TN, FN, FP = 0,0,0,0
for i in range(len(y_test)):
    if y_test.iloc[i] == predict[i]:
        if predict[i] == 'M':
            TP += 1
        else :
            TN += 1
    elif y_test.iloc[i] == 'M' and predict[i] == 'B' :
        FN += 1
    else :
        FP += 1

In [9]:
confusion_matrix = [[TP, FN], [FP, TN]]
confusion_matrix

[[42, 1], [1, 70]]

In [10]:
accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1])/len(y_test)
precision = confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[1][0])
recall = confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[0][1])
print(f'"Accuracy ={accuracy:.4}\tPrecision ={precision:.4f}\trecall ={recall:.4f}')

"Accuracy =0.9825	Precision =0.9767	recall =0.9767
